In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"
#r "nuget: XPlot.GoogleCharts, 3.0.1"

open MathNet.Numerics.LinearAlgebra
open XPlot.GoogleCharts

## Вариант 9

In [ ]:
let a = 0.
let b = 1.
let lambda = 0.1
let h = fun x y -> - lambda * sin (x * (0.5 + y ** 2.))
let f = fun x -> x + 0.1

In [ ]:
/// Отрезок [A; B]
type LineSegment = {
    Left: float
    Right: float
} with 
    static member Default = {
        Left = 0.
        Right = 1. 
    } 
    
    member this.Length = 
        this.Right - this.Left
        
    member this.Middle = 
        (this.Left + this.Right) / 2.

    override this.ToString () = 
        sprintf "[%0.2f , %0.2f]" this.Left this.Right

module Segment = 
    open System

    type SegmentType = 
        | WithRightEndpoint
        | WithLeftEndpoint
        | WithBothEndpoints
        | WithoutBothEndpoints

    let splitSegmentWithStep stepSize segment = 
        Seq.unfold 
            (fun leftBorder -> 
                if Math.Abs (leftBorder - segment.Right) < 1e-10 then None
                else
                    let rightBoarder = min segment.Right (leftBorder + stepSize)
                    Some ({Left = leftBorder; Right = rightBoarder}, rightBoarder)
            ) segment.Left

    let splitSegmentIntoEqualParts amountOfParts segment = 
        splitSegmentWithStep ((segment.Right - segment.Left) / (float)amountOfParts) segment

    let getEquidistantPoints sType amountOfParts (segment: LineSegment) = 
        let bothEndpoint = 
            splitSegmentIntoEqualParts amountOfParts segment
            |> Seq.map (fun segment -> segment.Right)
            |> Seq.toList
            |> (fun tail -> segment.Left :: tail)
        let withoutLeft = lazy bothEndpoint.Tail
        let withoutRight = lazy List.take (bothEndpoint.Length - 1) bothEndpoint
        let withoutBoth = lazy withoutRight.Value.Tail

        match sType with 
        | WithBothEndpoints -> bothEndpoint
        | WithLeftEndpoint -> withoutRight.Value
        | WithRightEndpoint -> withoutLeft.Value
        | WithoutBothEndpoints -> withoutBoth.Value

In [ ]:
open Segment

let solveQuadrature (h: float -> float -> float) f (segment: LineSegment) knotsCount= 
    let compositeTrapezoidalRule (segment: LineSegment) knotsCount = 
        let knots = segment |> Segment.getEquidistantPoints WithBothEndpoints (knotsCount - 1)
        let coeffs = 
            List.init knotsCount (fun i ->
                if i = 0 then (knots.[1] - knots.[0]) / 2.
                elif i = knotsCount - 1 then (knots.[knotsCount - 1] - knots.[knotsCount - 2]) / 2.
                else (knots.[i + 1] - knots.[i - 1]) / 2.
            )

        knots, coeffs

    let (knots, coeffs) = compositeTrapezoidalRule segment knotsCount

    let d = 
        DenseMatrix.init knotsCount knotsCount (fun i j ->
            (if i = j then 1. else 0.) - coeffs.[j] * h knots.[i] knots.[j]
        )
    let g = DenseVector.ofList knots |> Vector.map f
    let z = d.Solve g
    let u = fun x -> (Vector.sum <| DenseVector.init knotsCount (fun i -> coeffs.[i] * h x knots.[i] * z.[i])) + f x

    u, knots, coeffs

In [ ]:
open System.Collections.Generic

let doWhile body condition =
    body ()
    while condition () do
        body ()

type InerationTableRecord = {
    KnotsCount: int
    ValueInA: float
    ValueInMiddle: float
    ValueInB: float
}

let iterationTable = List<InerationTableRecord>()

let mutable n = 2
let middle = (a + b) / 2.
let epsilon = 0.000001
let taskSolver = solveQuadrature h f { Left = a; Right = b }

let (u, knots, coeffs) = taskSolver n
let mutable prevIteration = u

{
    KnotsCount = n
    ValueInA = prevIteration a
    ValueInMiddle = prevIteration middle
    ValueInB = prevIteration b
} 
|> iterationTable.Add

// [
//     List.init n (fun i -> i, knots.[i] :> value)
//     List.init n (fun i -> i, coeffs.[i] :> value)
//     List.init n (fun i -> i, u knots.[i] :> value)
// ]
// |> Chart.Table
// |> Chart.WithLabels ["Knot Index"; "Knot"; "Coeff"; "Value"]
// |> Chart.Show

let mutable currIteration = prevIteration

doWhile 
    <| fun () ->
        prevIteration <- currIteration

        n <- n * 2
        let (u, knots, coeffs) = taskSolver n
        currIteration <- u

        // [
        //     List.init n (fun i -> i, knots.[i] :> value)
        //     List.init n (fun i -> i, coeffs.[i] :> value)
        //     List.init n (fun i -> i, u knots.[i] :> value)
        // ]
        // |> Chart.Table
        // |> Chart.WithLabels ["Knot Index"; "Knot"; "Coeff"; "Value"]
        // |> Chart.Show

        {
            KnotsCount = n
            ValueInA = currIteration a
            ValueInMiddle = currIteration middle
            ValueInB = currIteration b
        }
        |> iterationTable.Add

    <| fun () -> 
        [
            currIteration a - prevIteration a
            currIteration middle - prevIteration middle
            currIteration b - prevIteration b
        ]
        |> List.map abs
        |> List.max >= epsilon

display iterationTable

[
    currIteration a - prevIteration a
    currIteration middle - prevIteration middle
    currIteration b - prevIteration b
]
|> List.map abs
|> List.max

index,KnotsCount,ValueInA,ValueInMiddle,ValueInB
0,2,0.1,0.5631316507830182,1.0454607780663607
1,4,0.1,0.5723679624966344,1.053999664259087
2,8,0.1,0.5732586500927012,1.054507667179724
3,16,0.1,0.5734139921437909,1.0545896445289982
4,32,0.1,0.5734470585075647,1.0546068399378103
5,64,0.1,0.5734547167073013,1.054610809671885
6,128,0.1,0.5734565611310649,1.0546117650386602
7,256,0.1,0.5734570138118867,1.0546119994738747


4.5268082182747804E-07